In [1]:
from QMLE_scipy import *
from explore import *

### 1. Test k=2

In [4]:
# set up initial guess
sigma = 1
lam = 0.5
gamma = 0.1
rho = 0.2
beta = np.array([1,2])
params = [sigma,lam,gamma, rho, *beta]
params

# set up input data
n = 4
T = 4
k = 2
alpha = np.random.normal(0,1,4)
c0 = np.random.normal(0,1,4)
x1 = []
for i in range(T):
    x1.append(np.random.normal(0,1,n*k).reshape(n,k))
y1 = []
for i in range(T+1):
    y1.append(np.random.normal(0,1,4))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,1,0,0]]).reshape(4,4)
w1 = np.array([[0,0,1,0],[1,0,0,0],[0,1,0,0],[1,0,0,0]]).reshape(4,4)
W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

In [5]:
# run optimization with constraints
QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params,est_coeff = False)

[sigma,lam,gamma,rho,beta] =  [0.5562986375400485, 0.9999999456513948, -0.07208162032924174, 1.1522568096806676, array([0.04571296, 0.33487399])]


[0.5562986375400485,
 0.9999999456513948,
 -0.07208162032924174,
 1.1522568096806676,
 array([0.04571296, 0.33487399])]

In [6]:
QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params,est_coeff = True)

[sigma,lam,gamma,rho,beta] =  [0.5562986375400485, 0.9999999456513948, -0.07208162032924174, 1.1522568096806676, array([0.04571296, 0.33487399])]
c0 :  [[-0.18627668]
 [-1.00509166]
 [-0.26689707]
 [-0.91207821]]
alpha :  [[ 1.46133067]
 [-1.06023144]
 [ 0.05896894]
 [-0.46006816]]
residuals:  [array([[-0.34078791],
       [-0.80150251],
       [ 0.72220285],
       [ 0.42008757]]), array([[ 0.0848689 ],
       [ 0.83534517],
       [-0.63505606],
       [-0.28515816]]), array([[ 0.47291341],
       [-0.60714595],
       [-0.00204753],
       [ 0.13628009]]), array([[-0.21699438],
       [ 0.57330321],
       [-0.08509932],
       [-0.27120956]])]
asymptotic variance:  [[ 5.07422360e-02 -1.28025371e-02  3.79989300e-03 -1.09875863e-02
   9.93700847e-02  1.68832643e-02]
 [-1.28025384e-02 -4.08683738e-01 -6.90506086e-02 -1.09174799e-01
  -1.28455372e+00 -2.34274303e-01]
 [ 3.79989274e-03 -6.90506078e-02  5.50952532e-02 -1.95227585e-03
  -2.01359466e-01 -3.70360245e-02]
 [-1.09875867e-02 -

[[0.5562986375400485,
  0.9999999456513948,
  -0.07208162032924174,
  1.1522568096806676,
  array([0.04571296, 0.33487399])],
 array([[-0.18627668],
        [-1.00509166],
        [-0.26689707],
        [-0.91207821]]),
 array([[ 1.46133067],
        [-1.06023144],
        [ 0.05896894],
        [-0.46006816]]),
 [array([[-0.34078791],
         [-0.80150251],
         [ 0.72220285],
         [ 0.42008757]]),
  array([[ 0.0848689 ],
         [ 0.83534517],
         [-0.63505606],
         [-0.28515816]]),
  array([[ 0.47291341],
         [-0.60714595],
         [-0.00204753],
         [ 0.13628009]]),
  array([[-0.21699438],
         [ 0.57330321],
         [-0.08509932],
         [-0.27120956]])],
 array([[ 5.07422360e-02, -1.28025371e-02,  3.79989300e-03,
         -1.09875863e-02,  9.93700847e-02,  1.68832643e-02],
        [-1.28025384e-02, -4.08683738e-01, -6.90506086e-02,
         -1.09174799e-01, -1.28455372e+00, -2.34274303e-01],
        [ 3.79989274e-03, -6.90506078e-02,  5.50952

In [7]:
QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params, est_coeff = False, constrain= False)

[sigma,lam,gamma,rho,beta] =  [0.5824330258837707, 0.9999999982967367, -0.1639870783956191, 0.8113999364967899, array([-0.05099548,  0.24170238])]


[0.5824330258837707,
 0.9999999982967367,
 -0.1639870783956191,
 0.8113999364967899,
 array([-0.05099548,  0.24170238])]

In [8]:
QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params, est_coeff = True, constrain= False)

[sigma,lam,gamma,rho,beta] =  [0.5824330258837707, 0.9999999982967367, -0.1639870783956191, 0.8113999364967899, array([-0.05099548,  0.24170238])]
c0 :  [[ 0.03934677]
 [-0.80823326]
 [-0.33572081]
 [-0.68385168]]
alpha :  [[ 1.01884983]
 [-0.80801465]
 [ 0.1222088 ]
 [-0.33304398]]
residuals:  [array([[-0.53948065],
       [-0.77568891],
       [ 1.02549883],
       [ 0.28967072]]), array([[ 0.22694145],
       [ 0.72298489],
       [-0.57672947],
       [-0.37319687]]), array([[ 0.57927375],
       [-0.45241383],
       [-0.27719996],
       [ 0.15034004]]), array([[-0.26673455],
       [ 0.50511785],
       [-0.1715694 ],
       [-0.06681389]])]
asymptotic variance:  [[ 5.86731638e-02 -7.15587106e-02 -5.05351138e-03 -2.70746029e-02
  -3.66459533e-03 -3.42255854e-04]
 [-7.15587110e-02  3.08565420e-01  4.33459343e-02  9.29757181e-02
  -4.66084660e-03  1.87285490e-04]
 [-5.05351134e-03  4.33459344e-02  7.90752223e-02  3.14639219e-02
  -6.01727704e-04 -7.07630798e-06]
 [-2.70746029e-02 

[[0.5824330258837707,
  0.9999999982967367,
  -0.1639870783956191,
  0.8113999364967899,
  array([-0.05099548,  0.24170238])],
 array([[ 0.03934677],
        [-0.80823326],
        [-0.33572081],
        [-0.68385168]]),
 array([[ 1.01884983],
        [-0.80801465],
        [ 0.1222088 ],
        [-0.33304398]]),
 [array([[-0.53948065],
         [-0.77568891],
         [ 1.02549883],
         [ 0.28967072]]),
  array([[ 0.22694145],
         [ 0.72298489],
         [-0.57672947],
         [-0.37319687]]),
  array([[ 0.57927375],
         [-0.45241383],
         [-0.27719996],
         [ 0.15034004]]),
  array([[-0.26673455],
         [ 0.50511785],
         [-0.1715694 ],
         [-0.06681389]])],
 array([[ 5.86731638e-02, -7.15587106e-02, -5.05351138e-03,
         -2.70746029e-02, -3.66459533e-03, -3.42255854e-04],
        [-7.15587110e-02,  3.08565420e-01,  4.33459343e-02,
          9.29757181e-02, -4.66084660e-03,  1.87285490e-04],
        [-5.05351134e-03,  4.33459344e-02,  7.9075

### 2. Test k=1

In [9]:
# set up initial guess
initial_guess = [1.5,0.2,0.5,0.3,1.2] # in the order sigma, lam, gamma, rho, beta (all real numbers)

n = 49
k = 1
T = 10

# generate samples
t_theta = [0.1,0.2,np.array([1]).reshape(k,1),0.5,1] # true parameters
t_gamma, t_rho, t_beta, t_lam, t_sig = t_theta
alpha = np.random.normal(0,1,T).reshape(T,1)
c0 = np.random.normal(0,1,n).reshape(n,1)
l_n = np.ones(n).reshape(n,1)
x = []
for i in range(T):
    x.append(np.random.normal(0,1,n).reshape(n,k))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = generate_weight_matrix(n)
w1 = generate_weight_matrix(n)

W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

        V_nt = []
for i in range(T):
    V_nt.append(np.random.normal(0,t_sig,n).reshape(n,1))

Y0 = np.random.normal(0,1,n).reshape(n,1)

y = []
y.append(Y0)
for i in range(T):
    c_vec = t_gamma*y[i]+t_rho*np.matmul(W_ls[i],np.array(y[i]).reshape(n,1))+np.matmul(x[i],np.array(t_beta).reshape(k,1))+c0+alpha[i][0]*l_n+V_nt[i]
    Y_nt = np.matmul(np.linalg.inv(np.identity(n)-t_lam*W_ls[i+1]),c_vec).reshape(n,1)
    y.append(Y_nt)

In [10]:
QMLE_scipy_estimate(x, y, W_ls,initial_guess = initial_guess, constrain= False, est_coeff = False)

[sigma,lam,gamma,rho,beta] =  [0.9248755225940917, 0.5093343054362643, 0.03394938012267523, 0.22136978542826433, array([0.9064247])]


[0.9248755225940917,
 0.5093343054362643,
 0.03394938012267523,
 0.22136978542826433,
 array([0.9064247])]

In [11]:
QMLE_scipy_estimate(x, y, W_ls,initial_guess = initial_guess, constrain= False, est_coeff = True)

[sigma,lam,gamma,rho,beta] =  [0.9248755225940917, 0.5093343054362643, 0.03394938012267523, 0.22136978542826433, array([0.9064247])]
c0 :  [[-0.0816051 ]
 [ 0.11203655]
 [ 0.90202221]
 [-0.44404952]
 [ 0.16902053]
 [-0.34002276]
 [-0.42796945]
 [-0.53850915]
 [-0.23601151]
 [ 0.48391661]
 [-0.90673024]
 [ 0.18755069]
 [ 0.02152149]
 [ 0.74814299]
 [ 0.77925423]
 [-0.64625631]
 [ 0.02773032]
 [ 0.02834947]
 [-0.26962334]
 [ 0.18858183]
 [ 0.17795872]
 [ 0.33409763]
 [-0.04836949]
 [ 1.02261149]
 [-0.56808126]
 [-0.44482265]
 [ 0.5012147 ]
 [-0.27080935]
 [ 0.35060147]
 [-0.28599536]
 [ 1.36630423]
 [-0.14700616]
 [ 0.76253184]
 [-0.64068602]
 [ 0.46573601]
 [ 0.01245989]
 [ 0.24236285]
 [-0.71708327]
 [-0.32997855]
 [-0.63906334]
 [-0.90543978]
 [ 0.56596823]
 [ 0.44748527]
 [ 0.18116617]
 [ 0.88530264]
 [-0.62524124]
 [-0.42050793]
 [ 0.03237742]
 [ 0.14837801]]
alpha :  [[ 0.75209389]
 [-0.35538215]
 [ 0.14228918]
 [-0.20849207]
 [ 0.24867511]
 [-0.2022915 ]
 [-0.30486357]
 [ 0.195231

[[0.9248755225940917,
  0.5093343054362643,
  0.03394938012267523,
  0.22136978542826433,
  array([0.9064247])],
 array([[-0.0816051 ],
        [ 0.11203655],
        [ 0.90202221],
        [-0.44404952],
        [ 0.16902053],
        [-0.34002276],
        [-0.42796945],
        [-0.53850915],
        [-0.23601151],
        [ 0.48391661],
        [-0.90673024],
        [ 0.18755069],
        [ 0.02152149],
        [ 0.74814299],
        [ 0.77925423],
        [-0.64625631],
        [ 0.02773032],
        [ 0.02834947],
        [-0.26962334],
        [ 0.18858183],
        [ 0.17795872],
        [ 0.33409763],
        [-0.04836949],
        [ 1.02261149],
        [-0.56808126],
        [-0.44482265],
        [ 0.5012147 ],
        [-0.27080935],
        [ 0.35060147],
        [-0.28599536],
        [ 1.36630423],
        [-0.14700616],
        [ 0.76253184],
        [-0.64068602],
        [ 0.46573601],
        [ 0.01245989],
        [ 0.24236285],
        [-0.71708327],
        [-0.3